In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch

from video_dataset.video import VideoFromVideoFile

from tas_helpers.visualization import SegmentationVisualizer

from bouldering_video_segmentation.models import VideoSegmentMlp
from bouldering_video_segmentation.extractors import ResNet3DFeatureExtractor

/Users/nadir/Documents/code/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/nadir/Documents/code/.venv/lib/python3.12/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Users/nadir/Documents/code/.venv/lib/python3.12/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


<div class="alert alert-warning">

**Information:** The constants below (`VIDEO_PATH` and `VIDEO_SEGMENT_MLP_MODEL_WEIGHTS_PATH`) are to be defined. You can find an example video and the models weights in the github repository.

</div>

In [ ]:
VIDEO_PATH = "..."
SEGMENT_SIZE = 32
NUMBER_OF_CLASSES = 5
VIDEO_SEGMENT_MLP_MODEL_WEIGHTS_PATH = "..."

In [ ]:
video_dir_path = "/".join(VIDEO_PATH.split("/")[:-1])
video_name, video_extension = VIDEO_PATH.split("/")[-1].split(".")

In [ ]:
extractor = ResNet3DFeatureExtractor()

model = VideoSegmentMlp(
    input_size=extractor.get_features_shape(),
    # NOTE: the model has been trained on 5 classes, thus the output size is 5 and can't be changed when used with the provided weights
    output_size=NUMBER_OF_CLASSES
)

model = model.load_state_dict(torch.load(VIDEO_SEGMENT_MLP_MODEL_WEIGHTS_PATH))

video = VideoFromVideoFile(
    videos_dir_path=video_dir_path,
    id=video_name,
    video_extension=video_extension
)

In [ ]:
predictions = []

for segment in video.get_segments(segment_size=SEGMENT_SIZE):
    features = extractor.transform_and_extract(segment)
    
    prediction = model(features)
    
    predictions.append(prediction)

In [ ]:
SegmentationVisualizer(segment, prediction).show()